In [1]:
!pip install langchain langchain-community langchain-text-splitters chromadb pypdf sentence-transformers transformers accelerate torch langchain_huggingface wikipedia faiss-cpu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 83.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 60.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 40.6 MB/s eta 0:00:00:00

In [2]:
from langchain_community.retrievers import WikipediaRetriever

In [3]:
retriever = WikipediaRetriever(top_k_results=2 , lang ="en")


In [4]:
query = "the geopolitical history of a india and pakistan from the perspective of a chinese"

docs = retriever.invoke(query)

In [5]:
for i,doc in enumerate(docs):
    print(f"\n --- Result {i+1}")
    print(f"Content : \n  {doc.page_content} ..")


 --- Result 1
Content : 
  The India–Pakistan war of 1971, also known as the third Indo-Pakistani war, was a military confrontation between India and Pakistan that occurred during the Bangladesh Liberation War in East Pakistan from 3 December 1971 until the Pakistani capitulation in Dhaka on 16 December 1971.  The war began with Pakistan's Operation Chengiz Khan, consisting of preemptive aerial strikes on eight Indian air stations. The strikes led to India declaring war on Pakistan, marking their entry into the war for East Pakistan's independence, on the side of Bengali nationalist forces. India's entry expanded the existing conflict with Indian and Pakistani forces engaging on both the eastern and western fronts.
Thirteen days after the war started, India achieved a clear upper hand, and the Eastern Command of the Pakistan military signed the instrument of surrender on 16 December 1971 in Dhaka, marking the formation of East Pakistan as the new nation of Bangladesh. Approximately 93

In [6]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings,HuggingFaceEndpoint
from langchain_core.documents import Document
from dotenv import load_dotenv
import os
load_dotenv()
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [8]:
vector_store = Chroma.from_documents(
    documents=documents,
    embedding = embedding,
    collection_name = "my_colection"
)

In [9]:
retriever = vector_store.as_retriever(search_kwargs={"k":2})

In [10]:
query = "What chroma is used for"
results = retriever.invoke(query)

In [11]:
for i,doc in enumerate(results):
    print(f"\n --- Result {i+1} --")
    print(doc.page_content)


 --- Result 1 --
Chroma is a vector database optimized for LLM-based search.

 --- Result 2 --
LangChain helps developers build LLM applications easily.


# MMR


In [12]:
 # Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [13]:
from langchain_community.vectorstores import FAISS

In [14]:
#embedding model above initialised
vector_store = FAISS.from_documents(
    documents=docs,
    embedding=embedding
)

In [15]:
retriever = vector_store.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k":3 , "lambda_mult" :0.5 }

)

In [16]:
query = "What is Langchain?"
results = retriever.invoke(query)

In [17]:
for i,doc in enumerate(results):
    print(f"\n -- Result {i+1} --")
    print(doc.page_content)


 -- Result 1 --
LangChain supports Chroma, FAISS, Pinecone, and more.

 -- Result 2 --
LangChain is used to build LLM based applications.

 -- Result 3 --
Embeddings are vector representations of text.


# Multiquery Retriever

## LangChain deprecated MultiQueryRetriever in favor of explicit query-generation pipelines using Runnables.

# CONTEXTUAL CPOMPRESSION RETRIEVER

In [18]:
from langchain_community.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever

ModuleNotFoundError: No module named 'langchain.retrievers'

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

ModuleNotFoundError: No module named 'langchain.retrievers'

In [ ]:
!pip uninstall -y langchain langchain-community langchain-core
!pip install -U langchain langchain-community

Found existing installation: langchain 1.2.0
Uninstalling langchain-1.2.0:
  Successfully uninstalled langchain-1.2.0
Found existing installation: langchain-community 0.4.1
Uninstalling langchain-community-0.4.1:
  Successfully uninstalled langchain-community-0.4.1
Found existing installation: langchain-core 1.2.5
Uninstalling langchain-core-1.2.5:
  Successfully uninstalled langchain-core-1.2.5
  Using cached langchain-1.2.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_core-1.2.5-py3-none-any.whl.metadata (3.7 kB)
Using cached langchain-1.2.0-py3-none-any.whl (102 kB)
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)
Using cached langchain_core-1.2.5-py3-none-any.whl (484 kB)


In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

ModuleNotFoundError: No module named 'langchain.retrievers'

In [19]:
import langchain
import langchain_community

print(langchain.__version__)
print(langchain_community.__version__)

1.2.0
0.4.1


In [42]:
import os

import chromadb
from langchain_classic.retrievers import (
    ContextualCompressionRetriever,
    MergerRetriever,
)
from langchain_classic.retrievers.document_compressors import LLMChainExtractor
from langchain_classic.retrievers.document_compressors import DocumentCompressorPipeline

from langchain_community.document_transformers import (
    EmbeddingsClusteringFilter,
    EmbeddingsRedundantFilter,
)
from langchain_huggingface import HuggingFaceEmbeddings,ChatHuggingFace,HuggingFaceEndpoint
from dotenv import load_dotenv
load_dotenv()




False

In [43]:
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [44]:
vectorStore = FAISS.from_documents(docs,embedding)

In [45]:
base_retieirever = vector_store.as_retriever(search_kwargs={"k":5})

In [46]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task = "text-generation"
)
model = ChatHuggingFace(llm=llm)

In [47]:
compressor = LLMChainExtractor.from_llm(llm)

In [48]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retieirever,
    base_compressor=compressor
)

In [49]:
query= "What is Photosynthesis?"
compressed_results = compression_retriever.invoke(query)

ValueError: You must provide an api_key to work with novita API or log in with `hf auth login`.

In [52]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from dotenv import load_dotenv
load_dotenv()
import os
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task = "text-generation"
)
model = ChatHuggingFace(llm=llm)

In [51]:
print(model.invoke("what is ai?"))

ValueError: You must provide an api_key to work with novita API or log in with `hf auth login`.